
# 1 Bootstrapping

In [56]:
import pandas as pd
from math import sqrt
import scipy.stats as st
data = pd.read_csv('SP 500.csv',sep=',',index_col=None,header=0)

In [57]:
data.head()

,Dates,SP500
0,1962-01-31,68.84
1,1962-02-28,69.96
2,1962-03-31,69.55
3,1962-04-30,65.24
4,1962-05-31,59.63


In [58]:
ColNames = data.columns.to_list()
print(ColNames)

['Dates', 'SP500']


In [59]:
price_index= data['SP500'].apply(lambda x: float(x))

In [60]:
import numpy as np
from statsmodels.tsa.stattools import acf

In [61]:
def VR1(x,k):
  #x = np.matrix(x).reshape(1,len(x))
  kvec=np.array([k])
  m = (acf(x,nlags=max(kvec)))[1:(max(kvec)+1)]
  VR = []
  w=[]
  for j in range(1,k+1):
        w.append(1-j/k)
  temp=np.multiply(np.array(w),np.array(m[0:k]))
  VR.append(1+ 2*sum(temp.tolist())) 
  return(VR)


In [62]:
def VR2(x,kvec):
  #x = np.matrix(x).reshape(1,len(x))
  #kvec=np.array([kvec])
    m = (acf(x,nlags=max(kvec)))[1:(max(kvec)+1)]
    VR = []
    for i in range(0,len(kvec)):
        k = kvec[i]
        w=[]
    for j in range(1,k+1):
        w.append(1-j/k)
    temp=np.multiply(np.array(w),np.array(m[0:k]))
    VR.append(1+ 2*sum(temp.tolist())) 
    return(VR)

In [63]:
def Lo_MacKinlay_stat(y,k):
    #y = np.matrix(y)
    y1 = (y-y.mean())**2
    n =y.shape[0]
    vr = VR1(y,k)
    tem1 = 2*(2*k-1)*(k-1)
    tem2 = 3*k
    m1=[]
    for i in range(0,len(vr)):
        m1.append(sqrt(n)*(vr[i]-1)/sqrt(tem1/tem2))
    w=[]
    for i in range(1,k):
        w.append(4*(1-i/k)**2) 
    dvec =[]
    for j in range(0,k-1):
        dvec.append(sum(np.multiply(np.array( y1[(j+1):n]),np.array(y1[1:(n-j)])))/( sum(y1)**2 ))
    summ = np.multiply(np.array(w),np.array(dvec))
    m2=[]
    for i in range(0,len(vr)):
        m2.append(sqrt(n)*(vr[i]-1)*((n*summ)**(-.5)))
    list1=[]
    list1.append(m1)
    list1.append(m2)
    return(list1)

Lo_MacKinlay_stat = function(y,k)
{
    y = as.matrix(y)
    y1 = (y-mean(y))^2
    n =nrow(y) 
    
    vr = VR(y,k)
    tem1 = 2*(2*k-1)*(k-1); tem2 <- 3*k
    m1 = sqrt(n)*(vr-1)/sqrt( tem1/tem2 )
    
    w = 4*as.matrix((1-(1:(k-1))/k)^2,nrow=k-1)
    dvec = matrix(NA, nrow=(k-1), ncol=1)
    for (j in 1:(k-1))
    { 
    dvec[j] = sum(y1[(j+1):n] * y1[1:(n-j)])/( sum(y1)^2 )
    }
    summ = crossprod(w,dvec)
    m2 = sqrt(n)*(vr-1)*((n*summ)^(-.5) )
return(list(LM1=m1,LM2=m2))
}

In [ ]:
Lo_MacKinlay_stat = function(y,k)
{
    y = as.matrix(y)
    y1 = (y-mean(y))^2
    n =nrow(y) 
    
    vr = VR(y,k)
    tem1 = 2*(2*k-1)*(k-1); tem2 <- 3*k
    m1 = sqrt(n)*(vr-1)/sqrt( tem1/tem2 )
    
    w = 4*as.matrix((1-(1:(k-1))/k)^2,nrow=k-1)
    dvec = matrix(NA, nrow=(k-1), ncol=1)
    for (j in 1:(k-1))
    { 
    dvec[j] = sum(y1[(j+1):n] * y1[1:(n-j)])/( sum(y1)^2 )
    }
    summ = crossprod(w,dvec)
    m2 = sqrt(n)*(vr-1)*((n*summ)^(-.5) )
return(list(LM1=m1,LM2=m2))
}

In [64]:
def LM_stat_vec(y,kvec):

    #y = as.matrix(y)
    n = y.shape[0]
    mq = np.empty((len(kvec),2))
    for i in range(0,len(kvec)):
        k = kvec[i]
        LM = Lo_MacKinlay_stat(y,k)
        mq[i][0]=LM[0][0]
        mq[i][1]=LM[1][0][0]
    list2=[]
    list2.append(mq[:,0])
    list2.append(mq[:,1])
    return(list2)

In [ ]:
LM_stat_vec = function(y,kvec)
{
    y = as.matrix(y)
    n = nrow(y)
    mq = matrix(NA, nrow=length(kvec), ncol=2)
    for (i in 1:length(kvec))
    {
    k = kvec[i]
    LM = Lo_MacKinlay_stat(y,k)
    mq[i,] = cbind(LM$LM1,LM$LM2)
    }
    
return(list(M1=mq[,1],M2=mq[,2]))
}

In [65]:
def Bootstrap(r,kvec,B,prob=[0.025,0.975]):
    np.random.seed(1)
    VR = LM_stat_vec(r,kvec)
    VR_mat = np.empty((B,len(kvec)))
    VR_stat =VR[1].tolist()
    p = np.empty((VR_mat.shape[1],1))
    CI = np.empty((VR_mat.shape[1],len(prob)))
    for i in range(1,B):
        rs = r * np.random.normal(0,1,len(r))
        VRs = LM_stat_vec(rs,kvec)
        for j in range(0,len(kvec)):
            VR_mat[i][j] = VRs[1][j]
    for i in range(0,len(kvec)):
        tem=[]
        for j in range(0,VR_mat.shape[0]):
            tem.append(abs(VR_mat[j,i]) > abs(VR_stat[i]))
        tem = [1 if m ==True else 0 for m in tem]
        p[i][0] = np.array(tem).mean()
        CI[i] = np.quantile(VR_mat[:,1],prob)
        
    CI = CI[0:len(kvec),]
    CI=pd.DataFrame(CI,columns=['2.5%','97.5%'],index=['k=2','k=4','k=8'])
    tinydict ={'Holding.Period':kvec,'VR_statistic':p[0:len(kvec)].tolist(),'CI':CI}
    return(tinydict)


In [ ]:
Bootstrap = function(r,k,B,prob=c(0.025,0.975))
{
    set.seed(666)
    r = as.matrix(r)
    VR = LM_stat_vec(r,kvec)
    
    VR_mat = matrix(NA, nrow=B, ncol=length(k))
    
    VR_stat = matrix(c(VR$M2))
    for (i in 1:B)
    {
    rs <- r * rnorm(nrow(r))
    VRs <- LM_stat_vec(rs,k)
    VR_mat[i,] <- c(VRs$M2)
    }
    
    p <- matrix(NA,nrow = ncol(VR_mat), ncol=1)
    CI <- matrix(NA,nrow = ncol(VR_mat), ncol=length(prob))
    for (i in 1:ncol(VR_mat))
        {
        tem <- abs(VR_mat[,i]) > abs(VR_stat[i])
        tem[tem == "TRUE"] <- 1
        p[i] <- mean(tem) 
        CI[i,] <- quantile(VR_mat[,i],prob)
        }
   CI <- CI[1:length(kvec),]
   colnames(CI) <- paste(100*prob,"%",sep="")
   rownames(CI) <- paste("k=",kvec,sep="")
         
return(list(Holding.Period=k,VR_statistic=as.numeric(p[1:length(k)]),CI=CI))
}

In [66]:
nob = len(price_index)
r=[]
for i in range(0,nob-1):
    r.append(np.log(price_index[i+1])-np.log(price_index[i]))
r=np.array(r)                             
kvec = [2,4,8] 
B = 1000

In [67]:
result=Bootstrap(r,kvec,1000,prob=[0.025,0.975])
for (key,value) in result.items():
    print (key,'\n',value,'\n')

Holding.Period 
 [2, 4, 8] 

VR_statistic 
 [[0.258], [0.68], [0.381]] 

CI 
          2.5%     97.5%
k=2 -0.885194  0.923748
k=4 -0.885194  0.923748
k=8 -0.885194  0.923748 



# 2 computed by usual asymptotic distribution

In [45]:
LM_stat_vec(r,kvec)

[array([0.9464672 , 0.30628207, 0.67569741]),
 array([0.40867417, 0.16494379, 0.49316228])]

array1 is the statistic under iid homoskedastic null hypothesis, M2 is the statistic under heteroskedastic null hypothesis. So, we will use M2 statistics here and compare.

In [46]:
def P_value(x,paramet,side=0):
    n = len(paramet)
    p = st.norm.cdf(x,paramet[0],paramet[1])       
    if side <0:
        return p
    elif side >0:
        return 1-p
    elif p< 0.5:
        return 2*p
    else :
        return 2*(1-p)



In [47]:
x=P_value(1.96,[0,1],0)
x

0.04999579029644097

This is the p value using the usual asymptotic distribution

In [50]:
asp = LM_stat_vec(r,kvec)[1]
p_list_asp =[]
for i in asp:
    p_list_asp.append(P_value(i,[0,1],0))
p_list_asp

[0.682778792652722, 0.8689882287080333, 0.6218979304967542]

This is the p value using bootstrapping

In [52]:
bs = Bootstrap(r,kvec,1000,prob=[0.025,0.975])['VR_statistic']
p_list_bs =[]
for i in bs:
    p_list_bs.append(P_value(i,[0,1],0)[0])

p_list_bs

[0.7964069072245956, 0.4965044609071412, 0.7032032485372492]

their p value are not the same. But the conclusion is the same, meaning that p values are larger than 0.05, rejecting the null hypothesis, which shows that the sp500 return series is not stationary and random walk. When q is small, the p value computed by two methods are quite different. However, when q becomes large, their p values are very close